In [73]:
# Imports
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from pytorch_metric_learning import losses, samplers
from skimage import io, transform
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, models, transforms

In [74]:
# Configurations
# Device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Num epochs
num_epochs = 25

# Model 
model = models.resnet50()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Batch size
batch_size = 32

# Data set
train_path = '/lab/vislab/DATA/CUB/images/'

# Loss function
criterion = losses.TripletMarginLoss(margin=0.1)

In [75]:
# Data Loader
transformations = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
])
dataset = datasets.ImageFolder(train_path, transformations)



# dataset_size = len(dataset)
# indices = list(range(dataset_size))
# split = int(np.floor(.4 * dataset_size))
# np.random.seed(32)
# np.random.shuffle(indices)
# train_indices, val_indices = indices[split:], indices[:split]


# test_split = int(np.floor(.1 * dataset_size))
# np.random.seed(32)
# np.random.shuffle(val_indices)
# val_indices, test_indices = val_indices[test_split:], val_indices[:test_split]

# # Creating PT data samplers and loaders:
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)
# test_sampler = SubsetRandomSampler(test_indices)

# train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
#                                            sampler=train_sampler, num_workers=4)
# validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                                 sampler=valid_sampler)
# test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                                 sampler=test_sampler)

In [76]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

def map_features(outputs, labels, out_file):
    # create array of column for each feature output
    feat_cols = ['feature'+str(i) for i in range(outputs.shape[1])]
    # make dataframe of outputs -> labels
    df = pd.DataFrame(outputs, columns=feat_cols)
    df['y'] = labels
    df['labels'] = df['y'].apply(lambda i: str(i))
    # clear outputs and labels
    outputs, labels = None, None
    # creates an array of random indices from size of outputs
    np.random.seed(42)
    rndperm = np.random.permutation(df.shape[0])
    num_examples = 3000
    df_subset = df.loc[rndperm[:num_examples],:].copy()
    data_subset = df_subset[feat_cols].values
    pca = PCA(n_components=50)
    pca_result = pca.fit_transform(data_subset)
    tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(data_subset)
    df_subset['tsne-2d-one'] = tsne_results[:,0]
    df_subset['tsne-2d-two'] = tsne_results[:,1]
    plt.figure(figsize=(16,10))
    plt.scatter(
        x=df_subset["tsne-2d-one"],
        y=df_subset["tsne-2d-two"],
        c=df_subset["y"],
        s=3
    )
    plt.savefig(out_file, bbox_inches='tight', pad_inches = 0)
    plt.close()


In [77]:
# Trainer
def train_model():
    """Generic function to train model"""

    start_time = datetime.now()
    correct = 0 
    incorrect = 0 
    num_batches = 0
    loss_values = []

    # Epochs 
    for epoch in range(num_epochs): 
        print("epoch num:", epoch)
        
        train_sampler = samplers.MPerClassSampler(dataset.targets, 8, len(dataset))
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers=4)
        
        running_outputs = torch.FloatTensor().cpu()
        running_labels = torch.LongTensor().cpu()
        running_loss = 0.0
        model.train()
        
        # Batches
        for batch_idx, (inputs, labels) in enumerate(train_loader):             
            num_batches += 1
            optimizer.zero_grad()

            inputs, labels = inputs.to(device), labels.to(device)
            output = model.forward(inputs)
            
            running_outputs = torch.cat((running_outputs, output.cpu().detach()), 0)
            running_labels = torch.cat((running_labels, labels.cpu().detach()), 0)

            loss = criterion(output, labels)
            loss = Variable(loss, requires_grad = True)
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        # Accuracy
        for idx, emb in enumerate(running_outputs.to(device)):    
            pairwise = torch.nn.PairwiseDistance(p=2).to(device)
            dist = pairwise(emb, running_outputs.to(device))
            closest = torch.topk(dist, 2, largest=False).indices[1]
            if running_labels[idx] == running_labels[closest]:
                correct += 1
            else:
                incorrect += 1

        running_outputs = torch.cat((running_outputs, output.cpu().detach()), 0)
        running_labels = torch.cat((running_labels, labels.cpu().detach()), 0)

        print(running_outputs.shape)
        print(running_labels.shape)
        print(running_loss / num_batches)
        print("correct", correct)
        print("incorrect", incorrect)

        # TSNE
        map_features(running_outputs, running_labels, "outfile")
        # Loss Plot            
        loss_values.append(running_loss / num_batches)

        
        time_elapsed = datetime.now() - start_time 
        print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))    

    plt.plot(loss_values)    
    return model, running_loss

In [78]:
def test():
    trained_model.eval()
    test_loss = 0.0
    with torch.no_grad():
        j = 0
        corrects = 0
        pairs = []
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = trained_model(data)
            
            print(output.shape)
            break
            
            # do we do any test loss
#             test_loss =  criterion(dista, distb, target).data.item()

            for row in output: 
                distance_matrix = accuracy(row)
                pairs.append(distance_matrix)
                print(pairs[0])
            d = distance_matrix.cpu().numpy()
            print(type(d))
            print(d.shape)
            
            print(min(d))

            
            # update losses? or no?

In [ ]:
# Run Script
model.to(device)

trained_model, loss = train_model()
print(loss)

epoch num: 0
torch.Size([4832, 1000])
torch.Size([4832])
0.15215656677881875
correct 428
incorrect 4372
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 3000 samples in 0.072s...
[t-SNE] Computed neighbors for 3000 samples in 11.808s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3000
[t-SNE] Computed conditional probabilities for sample 2000 / 3000
[t-SNE] Computed conditional probabilities for sample 3000 / 3000
[t-SNE] Mean sigma: 0.590418
[t-SNE] KL divergence after 250 iterations with early exaggeration: 67.102547
[t-SNE] KL divergence after 300 iterations: 1.743900
Time elapsed (hh:mm:ss.ms) 0:01:07.456657
epoch num: 1
torch.Size([4832, 1000])
torch.Size([4832])
0.07607828338940938
correct 856
incorrect 8744
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 3000 samples in 0.067s...
[t-SNE] Computed neighbors for 3000 samples in 11.571s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3000
[t-SNE] Computed conditional probabiliti